In [1]:
import os
import re
import geopy as gp
import numpy as np
import pandas as pd
import altair as alt
import geopandas as gpd
from Modules import preprocessing as pre

In [2]:
dir = './Data'
temp_pre = './Data/tmp_pre'
colission_exists = False

## Weather dataset preprocessing

In the process of downloading the data, we selected all the attributes available. Now, we will explore the data and select the ones that are useful for our purpose.

In [3]:
weather = pd.read_csv(f'{dir}/weather.csv')
weather.shape

C:\Users\jpniz\AppData\Local\Temp\ipykernel_10852\941678255.py:1: DtypeWarning: Columns (7,9,13,17,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65) have mixed types. Specify dtype option on import or set low_memory=False.
  weather = pd.read_csv(f'{dir}/weather.csv')


(90591, 66)

### Data exploration

In [4]:
weather.shape

(90591, 66)

Since we are only interested in the rows where the date is inside the timeranges of 01/06/2018 - 31/09/2018 and 01/06/2020 - 31/09/2020, we will filter the data to only include those rows.

In [5]:
weather = pre.time_filter(weather, 'DATE')

weather.to_csv(f'{dir}/weather_2018-2020.csv', index=False) # sobre el mismo archivo

In [6]:
weather.shape

(20536, 66)

In [7]:
weather.columns

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'AWND',
       'AWND_ATTRIBUTES', 'DAPR', 'DAPR_ATTRIBUTES', 'DASF', 'DASF_ATTRIBUTES',
       'MDPR', 'MDPR_ATTRIBUTES', 'MDSF', 'MDSF_ATTRIBUTES', 'PGTM',
       'PGTM_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'PSUN', 'PSUN_ATTRIBUTES',
       'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TAVG',
       'TAVG_ATTRIBUTES', 'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES',
       'TOBS', 'TOBS_ATTRIBUTES', 'TSUN', 'TSUN_ATTRIBUTES', 'WDF2',
       'WDF2_ATTRIBUTES', 'WDF5', 'WDF5_ATTRIBUTES', 'WESD', 'WESD_ATTRIBUTES',
       'WESF', 'WESF_ATTRIBUTES', 'WSF2', 'WSF2_ATTRIBUTES', 'WSF5',
       'WSF5_ATTRIBUTES', 'WT01', 'WT01_ATTRIBUTES', 'WT02', 'WT02_ATTRIBUTES',
       'WT03', 'WT03_ATTRIBUTES', 'WT04', 'WT04_ATTRIBUTES', 'WT05',
       'WT05_ATTRIBUTES', 'WT06', 'WT06_ATTRIBUTES', 'WT08', 'WT08_ATTRIBUTES',
       'WT09', 'WT09_ATTRIBUTES', 'WT11', 'WT11_ATTRIBUTES'],
      dtype='object')

By looking at the documentation of the weather datatset and the attributes present, each row represents some selected observations (values) available for a given **STATION** and **DATE**. Neither the **STATION** nor the **DATE** are unique.

In [8]:
len(weather['STATION'].unique()), len(weather['DATE'].unique())

(129, 244)

Apart from the first 6 columns listed avobe, the rest of the attributes correspond to optional flags and their respective attributes (definded by the weather documentation *Note: The 4 flags listed [...] are optional on the Custom GHCN-Daily ASCII Form*) and therefore can contain several null values. That is the reason for the large quantity of sparse attributes detected. We will explore the data to see which attributes are useful for our purpose.

All these atributes correspond to the Table 4. A brief description is collected in the following table:

| Attribute | Description |
| --- | --- |
| *PGTM* | Peak gust time (hours and minutes, i.e., HHMM) |
| *PRCP* | Precipitation (mm) |
| *SNOW* | Snowfall (mm) |
| *SNWD* | Snow depth (mm) |
| *TMAX* | Maximum temperature (Celsius) |
| *TMIN* | Minimum temperature (Celsius) |
| *TOBS* | Temperature at the time of observation |
| *AWND* | Average daily wind speed (meters per second) |

| *TAVG* | Average temperature (Celsius) |
| *TSUN* | Daily total sunshine (minutes) |
| *WSF** | Fastest *-minute wind speed (meters per second) |
| *WT** | Weather type * |

The next table contains the attributes that we considered that are not udeful for the visualization purpose:

| Attribute | Description |
| --- | --- |
| *DARP* | Number of days included in the multiday precipitation total (MDPR) |
| *DASF* | Number of days included in the multiday snowfall total (MDSF) |
| *MDPR* | Multiday precipitation total (mm; use with DAPR and DWPR, if available) |
| *MDSF* | Multiday snowfall total (mm; use with DASF and DWSF, if available) |
| *PSUN* | Daily percent of possible sunshine (percent; use with TSUN, if available) |
| *WDF** | Direction of fastest *-second wind (degrees) |
| *WESD* | Water equivalent of snow on the ground (decimal mm) |
| *WESF* | Water equivalent of snowfall (decimal mm) |


Both **DARP** and **DASF** doesn't directly measure specific weather conditions. They are more of an aggregate measure of precipitation/snowfall over multiple days, which is not useful for our purpose. The same applies to **MDPR**, **MDSF** and **PSUN** that are also not useful for our purpose since they are not specific weather conditions.

-- justificar WDF, WESD, WESF

The ***_ATTRIBUTES** columns refer to the rest of the tables in the documentation (Table 1, 2, 3). These tables serve as flags for the measurement, quality and source of the data respectively. The values available for these attributes cover a wide range. In the data selection process we will analyze the subset of those values that are present in the data and discuss if they are useful for our purpose.

### Data selection

#### Geographical attributes

First of all, we will start by joining **LATITUDE** and **LONGITUDE** in a **LOCATION** attribute following the format of the *collision* dataset. Since both attributes ahave no missing values, the resulting column will not have any missing values.

In [9]:
weather[(weather['LATITUDE'].notnull()) & (weather['LONGITUDE'].notnull())].shape

(20536, 66)

In [10]:
weather['LOCATION'] = '(' + weather['LATITUDE'].astype(str) + ', ' + weather['LONGITUDE'].astype(str) + ')'

We will delete the **ELEVATION** column since it can't be related in any way with the previous dataset and the information it gives is not useful for our visualization purpose.

In [11]:
weather.drop(columns=['LATITUDE', 'LONGITUDE', 'ELEVATION'], inplace=True)

cols = list(weather.columns)
cols = [cols[-1]] + cols[:-1]
weather = weather[cols]

Since we have *LOCATION*, we don't need *STATION* code or *NAME*.

-- yo no los quitaria aun, porque si queremos hacer un mapa de estaciones, necesitamos el codigo de estacion y el nombre de la estacion. Si no, no sabemos a que estacion corresponde cada punto.

In [12]:
weather.drop(columns=['STATION', 'NAME'], inplace=True)

#### Observational attributes

Since we are analyzing the summer periods of 2018 and 2020, it is unlikely to have observations of snow in the records. Under this assumption we can drop the **SNOW** and **SNWD** attributes.

In [13]:
weather['SNOW'].unique(), weather['SNWD'].unique()

(array([ 0., nan]), array([ 0., nan]))

As we supposed, the only observations registred of **SNOW** and **SNWD** are null values or 0. Therefore, we can drop these columns.

In [14]:
weather.drop(columns=['SNOW', 'SNWD', 'SNOW_ATTRIBUTES', 'SNWD_ATTRIBUTES'], inplace=True)

From the data exploration done in the previous section, we can remove the following attributes: **DARP**, **DASF**, **MDPR**, **MDSF**, **PSUN**, **WDF**, **WESD**, **WESF** and their respective flags.

In [15]:
weather.drop(columns=['DAPR',
                        'DASF',
                        'MDPR',
                        'MDSF',
                        'PSUN',
                        'WDF2',
                        'WDF5',
                        'WESD',
                        'WESF',
                        'WSF2',
                        'WSF5',
                        'DAPR_ATTRIBUTES',
                        'DASF_ATTRIBUTES',
                        'MDPR_ATTRIBUTES',
                        'MDSF_ATTRIBUTES',
                        'PSUN_ATTRIBUTES',
                        'WDF2_ATTRIBUTES',
                        'WDF5_ATTRIBUTES',
                        'WESD_ATTRIBUTES',
                        'WESF_ATTRIBUTES',
                        'WSF2_ATTRIBUTES',
                        'WSF5_ATTRIBUTES'
                        ], inplace=True)

weather.columns

Index(['LOCATION', 'DATE', 'AWND', 'AWND_ATTRIBUTES', 'PGTM',
       'PGTM_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'TAVG', 'TAVG_ATTRIBUTES',
       'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'TOBS',
       'TOBS_ATTRIBUTES', 'TSUN', 'TSUN_ATTRIBUTES', 'WT01', 'WT01_ATTRIBUTES',
       'WT02', 'WT02_ATTRIBUTES', 'WT03', 'WT03_ATTRIBUTES', 'WT04',
       'WT04_ATTRIBUTES', 'WT05', 'WT05_ATTRIBUTES', 'WT06', 'WT06_ATTRIBUTES',
       'WT08', 'WT08_ATTRIBUTES', 'WT09', 'WT09_ATTRIBUTES', 'WT11',
       'WT11_ATTRIBUTES'],
      dtype='object')

Regarding the **WT*** columns, they all describe a specific weather condition that could be referred as *adverse condition* given the descriptions from the documentation.


| Attribute | Description |
| --- | --- |
| *WT02* | Heavy fog or heaving freezing fog (not always distinguished from fog) |
| *WT03* | Thunder |
| *WT04* | Ice pellets, sleet, snow pellets, or small hail" |
| *WT05* | Hail (may include small hail) |
| *WT06* | Glaze or rime |
| *WT08* | Smoke or haze |
| *WT09* | Blowing or drifting snow |
| *WT11* | High or damaging winds |

As checked, they all have the null values in the same rows, which could indicate that the ``nan`` value could significate a ``False`` value for all the adverse type of weather conditions and meaning that the corresponding record refers to a *normal* weather day. Therefore, we will replace the ``nan`` values with ``0``.

In [16]:
weather[weather['WT01'].isnull() & weather['WT02'].isnull() & weather['WT03'].isnull() & weather['WT04'].isnull() & weather['WT05'].isnull() & weather['WT06'].isnull() & weather['WT08'].isnull() & weather['WT09'].isnull() & weather['WT11'].isnull()].shape

(19894, 36)

In [17]:
weather['WT01'].unique(), weather['WT02'].unique(), weather['WT03'].unique(), weather['WT04'].unique(), weather['WT05'].unique(), weather['WT06'].unique(), weather['WT08'].unique(), weather['WT09'].unique(), weather['WT11'].unique()

(array([nan,  1.]),
 array([nan,  1.]),
 array([nan,  1.]),
 array([nan]),
 array([nan,  1.]),
 array([nan]),
 array([nan,  1.]),
 array([nan]),
 array([nan,  1.]))

By knowing this, the final decision was to merge all the **WT*** columns into a single one called **ADVERSE CONDITION**, which would have boolean values (0 or 1) indicating if the day was normal or not.

In [18]:
wt_columns = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11']

weather['ADVERSE_CONDITIONS'] = weather[wt_columns].isnull().all(axis=1).astype(int)
weather['ADVERSE_CONDITIONS'] = 1 - weather['ADVERSE_CONDITIONS']

weather['ADVERSE_CONDITIONS'].value_counts()

ADVERSE_CONDITIONS
0    19894
1      642
Name: count, dtype: int64

In [19]:
wt_columns_att = [col + '_ATTRIBUTES' for col in wt_columns]

weather.drop(columns=wt_columns, inplace=True)
weather.drop(columns=wt_columns_att, inplace=True)

weather.columns

Index(['LOCATION', 'DATE', 'AWND', 'AWND_ATTRIBUTES', 'PGTM',
       'PGTM_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'TAVG', 'TAVG_ATTRIBUTES',
       'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'TOBS',
       'TOBS_ATTRIBUTES', 'TSUN', 'TSUN_ATTRIBUTES', 'ADVERSE_CONDITIONS'],
      dtype='object')

#### Flags attributes

The ***_ATTRIBUTES** have not been commented yet. 

-- mdatios sobre sensores y otras mierdas que no interesan

In [20]:
cols = weather.columns.tolist()

for col in cols:
    if '_ATTRIBUTES' in col:
        print(col, weather[col].unique(), sep=':')

AWND_ATTRIBUTES:[nan ',,W']
PGTM_ATTRIBUTES:[nan ',,W']
PRCP_ATTRIBUTES:[',,7,0700' 'T,,7,0700' ',,N' 'T,,N' nan ',,W,2400' 'T,,W,2400' ',L,N'
 ',,7,0800' ',,W' 'T,,W' ',,N,' 'T,,N,' ',,7,' ',,7,0630' 'T,,7,0630'
 ',,Z,0700']
TAVG_ATTRIBUTES:[nan 'H,,S']
TMAX_ATTRIBUTES:[',,7' nan ',,W' ',,Z']
TMIN_ATTRIBUTES:[',,7' nan ',,W']
TOBS_ATTRIBUTES:[',,7,0700' nan ',,7,0800' ',,7,' ',,7,0630']
TSUN_ATTRIBUTES:[nan]


In [21]:
wt_columns_att = [col for col in weather.columns if '_ATTRIBUTES' in col]

weather.drop(columns=wt_columns_att, inplace=True)

### Missing values

Based on the sparseness described in teh data exploration, it is logical to check for columns with only missing values in order to remove them.

In [22]:
weather.shape

(20536, 11)

Coincidentally, the **TSUN** and therefore **TSUN_ATTRIBUTES** columns have only missing values. Therefore, we will remove them.

In [23]:
weather.isnull().sum()

LOCATION                  0
DATE                      0
AWND                  18608
PGTM                  19560
PRCP                    297
TAVG                  19804
TMAX                  17221
TMIN                  17233
TOBS                  19244
TSUN                  20536
ADVERSE_CONDITIONS        0
dtype: int64

In [24]:
weather.drop(columns=['TSUN', 'PGTM'], inplace=True)

 For *PRCP*, we will assume that the null values are 0, that is if there's no record of precipitation (the value is null), it's because there were no precipitations. We make this assumption because the number of null values represents around 10% of the dataset so it's not such a risk to make this assumption (and even less risk taking into account that the data is from Summer, when it's less likely to rain).

In [25]:
weather['PRCP'].fillna(0.0, inplace=True)

Regarding temperature, the great number of missing values is a problem. We will impute the missing values with the avergae temperature per day of each attribute (*TMAX*, *TMIN*, *TOBS*).

In [26]:
weather['TMAX'] = weather.groupby('DATE')['TMAX'].transform(lambda x: x.fillna(x.mean()))
weather['TMIN'] = weather.groupby('DATE')['TMIN'].transform(lambda x: x.fillna(x.mean()))
weather['TOBS'] = weather.groupby('DATE')['TOBS'].transform(lambda x: x.fillna(x.mean()))


In [27]:
weather.drop(columns=['TAVG'], inplace=True)


In [28]:
weather.isnull().sum()

LOCATION                  0
DATE                      0
AWND                  18608
PRCP                      0
TMAX                      0
TMIN                      0
TOBS                      0
ADVERSE_CONDITIONS        0
dtype: int64

We will apply the same strategy on AWND to get rid of the missing values.

In [29]:
weather['AWND'] = weather.groupby('DATE')['AWND'].transform(lambda x: x.fillna(x.mean()))

In [30]:
weather.isnull().sum()

LOCATION              0
DATE                  0
AWND                  0
PRCP                  0
TMAX                  0
TMIN                  0
TOBS                  0
ADVERSE_CONDITIONS    0
dtype: int64

In [31]:
# Podemos categorizar AWND con la Beaufort scale (https://en.wikipedia.org/wiki/Beaufort_scale)

# Podemos categorizar PRCP con respecto a la World Meteorological Organization, 2018(https://www.researchgate.net/figure/Rain-intensity-classifications-according-to-the-World-Meteorological-Organization-2018_tbl1_353769617)


### Preprocessed data saving

In [32]:
weather.to_csv(f'{dir}/weather_clean.csv', index=False)